<h1>Monish Shah Code</h1>

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

In [47]:
train_data = pd.read_csv('C:\\Masters Data Science\\Datasets\\training_set_features.csv')
train_labels = pd.read_csv('C:\\Masters Data Science\\Datasets\\training_set_labels.csv')
test_data = pd.read_csv('C:\\Masters Data Science\\Datasets\\test_set_features.csv')

merged_data = pd.merge(train_data, train_labels, on="respondent_id", how='inner')

Filling Missing Values in Categorical features

In [48]:
#merged_data.loc[(merged_data['employment_status'] == 'Unemployed'), ['employment_industry', 'employment_occupation']] = ['Unemployed', 'Unemployed']
#test_data.loc[(test_data['employment_status'] == 'Unemployed'), ['employment_industry', 'employment_occupation']] = ['Unemployed', 'Unemployed']

#merged_data.loc[(merged_data['employment_status'] == 'Not in Labor Force'), ['employment_industry', 'employment_occupation']] = ['Not in Labor Force', 'Not in Labor Force']
#test_data.loc[(test_data['employment_status'] == 'Not in Labor Force') , ['employment_industry', 'employment_occupation']] = ['Not in Labor Force', 'Not in Labor Force']

#merged_data.loc[(merged_data['age_group'].isin(['65+ Years'])) & (merged_data['employment_status'].isnull()), ['employment_status']] = 'Not in Labor Force'
#test_data.loc[(test_data['age_group'].isin(['65+ Years'])) & (test_data['employment_status'].isnull()), ['employment_status']] = 'Not in Labor Force'

merged_data.loc[(merged_data['employment_occupation'].isnull() & (merged_data['employment_status'] == 'Unemployed')), 'employment_occupation'] = 'occupationUemp'
merged_data.loc[(merged_data['employment_industry'].isnull() & (merged_data['employment_status'] == 'Unemployed')), 'employment_industry'] = 'industryUemp'
test_data.loc[(test_data['employment_occupation'].isnull() & (test_data['employment_status'] == 'Unemployed')), 'employment_occupation'] = 'occupationUemp'
test_data.loc[(test_data['employment_industry'].isnull() & (test_data['employment_status'] == 'Unemployed')), 'employment_industry'] = 'industryUemp'

merged_data.loc[(merged_data['employment_industry'].isnull() & (merged_data['employment_status'] == 'Not in Labor Force')), 'employment_industry'] = 'industryna'
merged_data.loc[(merged_data['employment_occupation'].isnull() & (merged_data['employment_status'] == 'Not in Labor Force')), 'employment_occupation'] = 'occupationna'
test_data.loc[(test_data['employment_industry'].isnull() & (test_data['employment_status'] == 'Not in Labor Force')), 'employment_industry'] = 'industryna'
test_data.loc[(test_data['employment_occupation'].isnull() & (test_data['employment_status'] == 'Not in Labor Force')), 'employment_occupation'] = 'occupationna'

merged_data.loc[(merged_data['employment_industry'] == 'Unemployed') & (merged_data['employment_status'].isnull()), 'employment_industry'] = 'industryUemp'
merged_data.loc[(merged_data['employment_industry'] == 'Unemployed') & (merged_data['employment_status'].isnull()), 'employment_occupation'] = 'occupationUemp'
test_data.loc[(test_data['employment_industry'] == 'Unemployed') & (test_data['employment_status'].isnull()), 'employment_industry'] = 'industryUemp'
test_data.loc[(test_data['employment_industry'] == 'Unemployed') & (test_data['employment_status'].isnull()), 'employment_occupation'] = 'occupationUemp'

categorical_cols = ['education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'census_msa','employment_status','hhs_geo_region','age_group','employment_industry',
                    'employment_occupation']

merged_data[categorical_cols] = merged_data[categorical_cols].fillna('Missing')
test_data[categorical_cols] = test_data[categorical_cols].fillna('Missing')

Label encoding the Categorical and Ordinal Features

In [49]:
le = LabelEncoder()

cols_to_encode = ['education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'census_msa','employment_status','hhs_geo_region','age_group','employment_industry','employment_occupation']

merged_data[cols_to_encode] = merged_data[cols_to_encode].apply(le.fit_transform)
test_data[cols_to_encode] = test_data[cols_to_encode].apply(le.fit_transform)


Filling Missing values in the remaining numerical features

In [50]:
# define imputation function
def fill_missing_values_mean(df, cols):
    imputer = SimpleImputer(strategy='mean')
    df[cols] = imputer.fit_transform(df[cols])
    return df

# specify columns to impute
cols_to_impute_mean = ['h1n1_concern', 'h1n1_knowledge','opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 
                  'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 
                  'opinion_seas_sick_from_vacc']

# fill missing values with imputation for specified columns '', 
merged_data = fill_missing_values_mean(merged_data, cols_to_impute_mean)
test_data = fill_missing_values_mean(test_data, cols_to_impute_mean)

# define imputation function
#def fill_missing_values_mean(df, cols):
#    imputer = SimpleImputer(strategy='most_frequent')
#    df[cols] = imputer.fit_transform(df[cols])
#    return df

# specify columns to impute
#cols_to_impute_mean = ['behavioral_antiviral_meds', 
#                  'behavioral_avoidance',  'behavioral_face_mask',  'behavioral_wash_hands',  'behavioral_large_gatherings',  
#                  'behavioral_outside_home',  'behavioral_touch_face','household_adults', 'household_children']

# fill missing values with imputation for specified columns '', 
#merged_data = fill_missing_values_mean(merged_data, cols_to_impute_mean)
#test_data = fill_missing_values_mean(test_data, cols_to_impute_mean)

# define imputation function
def fill_missing_values(df, cols):
    imputer = SimpleImputer(strategy='constant', fill_value=-1)
    df[cols] = imputer.fit_transform(df[cols])
    return df

# specify columns to impute
cols_to_impute = ['health_insurance', 'behavioral_antiviral_meds', 
                  'behavioral_avoidance',  'behavioral_face_mask',  'behavioral_wash_hands',  'behavioral_large_gatherings',  
                  'behavioral_outside_home',  'behavioral_touch_face',
                  'doctor_recc_h1n1', 'doctor_recc_seasonal', 'chronic_med_condition', 
                  'child_under_6_months', 'health_worker','household_adults', 'household_children']

# fill missing values with imputation for specified columns '', 
merged_data = fill_missing_values(merged_data, cols_to_impute)
test_data = fill_missing_values(test_data, cols_to_impute)

df_train = merged_data
df_test = test_data

Other methods tried to Fill missing values

In [51]:
#categorical_cols = ['education',
# 'race',
# 'sex',
# 'income_poverty',
# 'marital_status',
# 'rent_or_own',
# 'census_msa','employment_status','hhs_geo_region','age_group','employment_industry',
#                    'employment_occupation']
#
#merged_data[categorical_cols] = merged_data[categorical_cols].fillna('Missing')
#test_data[categorical_cols] = test_data[categorical_cols].fillna('Missing')

In [52]:
# define imputation function
#def fill_missing_values(df, cols):
#    imputer = SimpleImputer(strategy='most_frequent')
#    df[cols] = imputer.fit_transform(df[cols])
#    return df

# specify columns to impute
#cols_to_impute = ['employment_industry','employment_occupation','employment_status']

# fill missing values with imputation for specified columns '',
#merged_data = fill_missing_values(merged_data, cols_to_impute)
#test_data = fill_missing_values(test_data, cols_to_impute)

In [53]:
# Perform BayesianRidge model-based imputation
#imputer_merged_data = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)
#imputer_test_data = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)

# Impute the missing values in the training data and convert it back to a DataFrame
#merged_data_imputed = imputer_merged_data.fit_transform(merged_data)
#merged_data_imputed_df = pd.DataFrame(merged_data_imputed, columns=merged_data.columns, index=merged_data.index)

# Impute the missing values in the test data using the same imputer and convert it back to a DataFrame
#test_data_imputed = imputer_test_data.fit_transform(test_data)
#test_data_imputed_df = pd.DataFrame(test_data_imputed, columns=test_data.columns, index=test_data.index)

In [54]:
# Calculate the percentage of respondents with health insurance for each employment status and income level
#docc_recc_pct_h1n1 = merged_data.groupby(['chronic_med_condition'])['doctor_recc_h1n1'].transform('mean')
#docc_recc_pct_seasonal = merged_data.groupby(['chronic_med_condition'])['doctor_recc_seasonal'].transform('mean')

# Fill in missing values for health insurance based on the respondent's employment status and income level
#merged_data['doctor_recc_h1n1'] = merged_data['doctor_recc_h1n1'].fillna(docc_recc_pct_h1n1)
#test_data['doctor_recc_h1n1'] = test_data['doctor_recc_h1n1'].fillna(docc_recc_pct_h1n1)
#merged_data['doctor_recc_seasonal'] = merged_data['doctor_recc_seasonal'].fillna(docc_recc_pct_seasonal)
#test_data['doctor_recc_seasonal'] = test_data['doctor_recc_seasonal'].fillna(docc_recc_pct_seasonal)

In [55]:
# Select relevant features for imputation
#features = ['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months', 'health_worker', 'age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'household_adults', 'household_children', 'employment_industry', 'employment_occupation']
#
# Perform KNN imputation
#imputer = KNNImputer(n_neighbors=5)
#merged_data[features] = imputer.fit_transform(merged_data[features])

# define imputation function
#def fill_missing_values(df, cols):
#    imputer = SimpleImputer(strategy='constant', fill_value=-1)
#    df[cols] = imputer.fit_transform(df[cols])
#    return df

# specify columns to impute
#cols_to_impute = ['health_insurance','h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds', 'behavioral_avoidance',  'behavioral_face_mask',  'behavioral_wash_hands',  'behavioral_large_gatherings',  'behavioral_outside_home',  'behavioral_touch_face', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc','doctor_recc_h1n1', 'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months', 'health_worker','household_adults', 'household_children']

# fill missing values with imputation for specified columns '',
#merged_data = fill_missing_values(merged_data, cols_to_impute)
#est_data = fill_missing_values(test_data, cols_to_impute)

In [56]:
# Replace missing values with NaN
#merged_data = merged_data.replace('?', np.nan)

# Fill in missing values in the categorical column based on the related columns
#mode_by_columns_roo = merged_data.groupby(['income_poverty', 'marital_status'])['rent_or_own'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['rent_or_own'] = merged_data.apply(lambda row: mode_by_columns_roo.get((row['income_poverty'], row['marital_status']), np.nan) if pd.isnull(row['rent_or_own']) else row['rent_or_own'], axis=1)
#test_data['rent_or_own'] = test_data.apply(lambda row: mode_by_columns_roo.get((row['income_poverty'], row['marital_status']), np.nan) if pd.isnull(row['rent_or_own']) else row['rent_or_own'], axis=1)

#mode_by_columns_ip = merged_data.groupby(['rent_or_own', 'marital_status'])['income_poverty'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['income_poverty'] = merged_data.apply(lambda row: mode_by_columns_ip.get((row['rent_or_own'], row['marital_status']), np.nan) if pd.isnull(row['income_poverty']) else row['income_poverty'], axis=1)
#test_data['income_poverty'] = test_data.apply(lambda row: mode_by_columns_ip.get((row['rent_or_own'], row['marital_status']), np.nan) if pd.isnull(row['income_poverty']) else row['income_poverty'], axis=1)

#mode_by_columns_ms = merged_data.groupby(['income_poverty', 'rent_or_own'])['marital_status'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['marital_status'] = merged_data.apply(lambda row: mode_by_columns_ms.get((row['income_poverty'], row['rent_or_own']), np.nan) if pd.isnull(row['marital_status']) else row['marital_status'], axis=1)
#test_data['marital_status'] = test_data.apply(lambda row: mode_by_columns_ms.get((row['income_poverty'], row['rent_or_own']), np.nan) if pd.isnull(row['marital_status']) else row['marital_status'], axis=1)

# Replace remaining NaN values with the mode of the entire column
#merged_data['rent_or_own'].fillna(merged_data['rent_or_own'].mode()[0], inplace=True)
#merged_data['income_poverty'].fillna(merged_data['income_poverty'].mode()[0], inplace=True)
#merged_data['marital_status'].fillna(merged_data['marital_status'].mode()[0], inplace=True)
#test_data['rent_or_own'].fillna(test_data['rent_or_own'].mode()[0], inplace=True)
#test_data['income_poverty'].fillna(test_data['income_poverty'].mode()[0], inplace=True)
#test_data['marital_status'].fillna(test_data['marital_status'].mode()[0], inplace=True)

# Fill in missing values in the remaining categorical columns based on the related columns
#mode_by_columns_edu = merged_data.groupby(['employment_status', 'employment_occupation'])['education'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['education'] = merged_data.apply(lambda row: mode_by_columns_edu.get((row['employment_status'], row['employment_occupation']), np.nan) if pd.isnull(row['education']) else row['education'], axis=1)
#test_data['education'] = test_data.apply(lambda row: mode_by_columns_edu.get((row['employment_status'], row['employment_occupation']), np.nan) if pd.isnull(row['education']) else row['education'], axis=1)

#mode_by_columns_emp = merged_data.groupby(['marital_status', 'age_group'])['employment_status'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['employment_status'] = merged_data.apply(lambda row: mode_by_columns_emp.get((row['marital_status'], row['age_group']), np.nan) if pd.isnull(row['employment_status']) else row['employment_status'], axis=1)
#test_data['employment_status'] = test_data.apply(lambda row: mode_by_columns_emp.get((row['marital_status'], row['age_group']), np.nan) if pd.isnull(row['employment_status']) else row['employment_status'], axis=1)

#mode_by_columns_ind = merged_data.groupby(['age_group', 'employment_status'])['employment_industry'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['employment_industry'] = merged_data.apply(lambda row: mode_by_columns_ind.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_industry']) else row['employment_industry'], axis=1)
#test_data['employment_industry'] = test_data.apply(lambda row: mode_by_columns_ind.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_industry']) else row['employment_industry'], axis=1)

#mode_by_columns_occ = merged_data.groupby(['age_group', 'employment_status'])['employment_occupation'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['employment_occupation'] = merged_data.apply(lambda row: mode_by_columns_occ.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_occupation']) else row['employment_occupation'], axis=1)
#test_data['employment_occupation'] = test_data.apply(lambda row: mode_by_columns_occ.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_occupation']) else row['employment_occupation'], axis=1)

#mode_by_columns_edu = merged_data.groupby(['employment_status', 'employment_occupation'])['education'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['education'] = merged_data.apply(lambda row: mode_by_columns_edu.get((row['employment_status'], row['employment_occupation']), np.nan) if pd.isnull(row['education']) else row['education'], axis=1)
#test_data['education'] = test_data.apply(lambda row: mode_by_columns_edu.get((row['employment_status'], row['employment_occupation']), np.nan) if pd.isnull(row['education']) else row['education'], axis=1)

#mode_by_columns_emp = merged_data.groupby(['marital_status', 'age_group'])['employment_status'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['employment_status'] = merged_data.apply(lambda row: mode_by_columns_emp.get((row['marital_status'], row['age_group']), np.nan) if pd.isnull(row['employment_status']) else row['employment_status'], axis=1)
#test_data['employment_status'] = test_data.apply(lambda row: mode_by_columns_emp.get((row['marital_status'], row['age_group']), np.nan) if pd.isnull(row['employment_status']) else row['employment_status'], axis=1)

#mode_by_columns_ind = merged_data.groupby(['age_group', 'employment_status'])['employment_industry'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['employment_industry'] = merged_data.apply(lambda row: mode_by_columns_ind.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_industry']) else row['employment_industry'], axis=1)
#test_data['employment_industry'] = test_data.apply(lambda row: mode_by_columns_ind.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_industry']) else row['employment_industry'], axis=1)

#mode_by_columns_occ = merged_data.groupby(['age_group', 'employment_status'])['employment_occupation'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
#merged_data['employment_occupation'] = merged_data.apply(lambda row: mode_by_columns_occ.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_occupation']) else row['employment_occupation'], axis=1)
#test_data['employment_occupation'] = test_data.apply(lambda row: mode_by_columns_occ.get((row['age_group'], row['employment_status']), np.nan) if pd.isnull(row['employment_occupation']) else row['employment_occupation'], axis=1)

# Replace remaining NaN values with the mode of the entire column
#merged_data['education'].fillna(merged_data['education'].mode()[0], inplace=True)
#merged_data['employment_status'].fillna(merged_data['employment_status'].mode()[0], inplace=True)
#merged_data['household_adults'].fillna(merged_data['household_adults'].mode()[0], inplace=True)
#merged_data['household_children'].fillna(merged_data['household_children'].mode()[0], inplace=True)
#merged_data['employment_industry'].fillna(merged_data['employment_industry'].mode()[0], inplace=True)
#merged_data['employment_occupation'].fillna(merged_data['employment_occupation'].mode()[0], inplace=True)
#test_data['education'].fillna(test_data['education'].mode()[0], inplace=True)
#test_data['employment_status'].fillna(test_data['employment_status'].mode()[0], inplace=True)
#test_data['employment_industry'].fillna(test_data['employment_industry'].mode()[0], inplace=True)
#test_data['household_adults'].fillna(test_data['household_adults'].mode()[0], inplace=True)
#test_data['household_children'].fillna(test_data['household_children'].mode()[0], inplace=True)
#test_data['employment_occupation'].fillna(test_data['employment_occupation'].mode()[0], inplace=True)

Creating new features

In [57]:
efficacy_cols = ['opinion_h1n1_vacc_effective', 'opinion_seas_vacc_effective']
df_train['vaccine_efficacy_mean'] = df_train[efficacy_cols].mean(axis=1)
df_test['vaccine_efficacy_mean'] = df_test[efficacy_cols].mean(axis=1)

df_train['h1n1_concern_knowledge_interaction'] = df_train['h1n1_concern'] + df_train['h1n1_knowledge']
df_test['h1n1_concern_knowledge_interaction'] = df_test['h1n1_concern'] + df_test['h1n1_knowledge']

#def combine_columns(row):
#    if row['doctor_recc_h1n1'] == -1 or row['vaccine_efficacy_mean'] == -1:
#        h1n1_combined = -1
#    else:
#        h1n1_combined = row['doctor_recc_h1n1'] * row['vaccine_efficacy_mean']
#        
#    if row['doctor_recc_seasonal'] == -1 or row['vaccine_efficacy_mean'] == -1:
#        seasonal_combined = -1
#    else:
#        seasonal_combined = row['doctor_recc_seasonal'] * row['vaccine_efficacy_mean']
#        
#    return pd.Series([h1n1_combined, seasonal_combined])
#
#df_train[['h1n1_combined', 'seasonal_combined']] = df_train.apply(combine_columns, axis=1)
#df_test[['h1n1_combined', 'seasonal_combined']] = df_test.apply(combine_columns, axis=1)


def create_missing_indicator_columns(df, cols):
    for col in cols:
        df[col + '_missing'] = df[col].isna().astype(int)
    return df

cols_to_check = ['health_worker','employment_occupation', 'employment_industry', 'health_insurance','employment_status','income_poverty','doctor_recc_h1n1', 'doctor_recc_seasonal']  # Replace with your actual column names

# Create the missing indicator columns for both merged_data and test_data
merged_data = create_missing_indicator_columns(merged_data, cols_to_check)
test_data = create_missing_indicator_columns(test_data, cols_to_check)

Other new features that were created but did not improve the model (Different Combinations were tried)

In [58]:
#df_train['doctor_recc_h1n1'] = df_train['doctor_recc_h1n1'].astype(bool)
#df_train['doctor_recc_seasonal'] = df_train['doctor_recc_seasonal'].astype(bool)
#df_train['doctor_recc'] = (df_train['doctor_recc_h1n1'] | df_train['doctor_recc_seasonal']).astype(int)
#df_test['doctor_recc_h1n1'] = df_test['doctor_recc_h1n1'].astype(bool)
#df_test['doctor_recc_seasonal'] = df_test['doctor_recc_seasonal'].astype(bool)
#df_test['doctor_recc'] = (df_test['doctor_recc_h1n1'] | df_test['doctor_recc_seasonal']).astype(int)

#df_train['doctor_recc_efficacy_interaction'] = df_train['doctor_recc'] * df_train['vaccine_efficacy_mean']
#df_test['doctor_recc_efficacy_interaction'] = df_test['doctor_recc'] * df_test['vaccine_efficacy_mean']

#df_train['h1n1_concern_knowledge_interaction'] = df_train['h1n1_concern'] * df_train['h1n1_knowledge']
#df_test['h1n1_concern_knowledge_interaction'] = df_test['h1n1_concern'] * df_test['h1n1_knowledge']

# New binary features
#opinion_threshold = 3
#binary_opinion_cols = ['opinion_h1n1_vacc_effective', 'opinion_seas_vacc_effective', 'opinion_h1n1_risk', 'opinion_seas_risk']

#for col in binary_opinion_cols:
#    new_col_name = f"high_{col}_belief"
#    df_train[new_col_name] = (df_train[col] >= opinion_threshold).astype(int)
#    df_test[new_col_name] = (df_test[col] >= opinion_threshold).astype(int)

#def create_missing_indicator_columns(df, cols):
#    for col in cols:
#        df[col + '_missing'] = df[col].isna().astype(int)
#    return df
# List of columns for which you want to create missing value indicators
#cols_to_check = ['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc','doctor_recc_h1n1', 'doctor_recc_seasonal']  # Replace with your actual column names

# Create the missing indicator columns for both merged_data and test_data
#merged_data = create_missing_indicator_columns(merged_data, cols_to_check)
#test_data = create_missing_indicator_columns(test_data, cols_to_check)
#merged_data['employment_type'] = merged_data['employment_industry'] + '_' + merged_data['employment_occupation']
#test_data['employment_type'] = test_data['employment_industry'] + '_' + test_data['employment_occupation']

# New features based on household composition
#df_train['household_net'] = df_train['household_adults'] + df_train['household_children'] * 0.5
#df_test['household_net'] = df_test['household_adults'] + df_test['household_children'] *0.5
#df_train['adult_to_child_ratio'] = df_train['household_adults'] / (df_train['household_children'] + 1)
#df_test['adult_to_child_ratio'] = df_test['household_adults'] / (df_test['household_children'] + 1)

# New binary features
#opinion_threshold = 3
#binary_opinion_cols = ['opinion_h1n1_vacc_effective', 'opinion_seas_vacc_effective', 'opinion_h1n1_risk', 'opinion_seas_risk']
#for col in binary_opinion_cols:
#    new_col_name = f"high_{col}_belief"
#    df_train[new_col_name] = (df_train[col] >= opinion_threshold).astype(int)
#    df_test[new_col_name] = (df_test[col] >= opinion_threshold).astype(int)

#df_train['Behavioral Precautions'] = df_train.iloc[:, 3:10].sum(axis=1)
#df_train.drop(df_train.columns[3:10], axis=1, inplace=True)

#df_train['age_chronic_condition'] = df_train['age_group'] * df_train['chronic_med_condition']
#df_test['age_chronic_condition'] = df_test['age_group'] * df_test['chronic_med_condition']

#df_train['health_worker_opinion_h1n1_risk'] = df_train['health_worker'] * df_train['opinion_h1n1_risk']
#df_train['health_worker_opinion_seas_risk'] = df_train['health_worker'] * df_train['opinion_seas_risk']
#df_test['health_worker_opinion_h1n1_risk'] = df_test['health_worker'] * df_test['opinion_h1n1_risk']
#df_test['health_worker_opinion_seas_risk'] = df_test['health_worker'] * df_test['opinion_seas_risk']

#df_train['age_health_insurance'] = df_train['age_group'] * df_train['health_insurance']
#df_test['age_health_insurance'] = df_test['age_group'] * df_test['health_insurance']

# Interaction between demographic features and opinions/behaviors
#demo_cols = ['income_poverty', 'employment_status', 'education']
#opinion_cols = ['opinion_h1n1_vacc_effective', 'opinion_seas_vacc_effective', 'opinion_h1n1_risk', 'opinion_seas_risk']
#for demo_col in demo_cols:
#    for opinion_col in opinion_cols:
#        new_col_name = f"{demo_col}_{opinion_col}_interaction"
#        df_train[new_col_name] = df_train[demo_col] * df_train[opinion_col]
#        df_test[new_col_name] = df_test[demo_col] * df_test[opinion_col]


Splitting the data

In [59]:
df_train = df_train.drop(['respondent_id'],axis=1)
X_test = df_test.drop(['respondent_id'], axis=1)

# Split data into X and y
X_train = df_train.drop(['h1n1_vaccine', 'seasonal_vaccine'],axis=1)
y_train_h1n1 = df_train['h1n1_vaccine']
y_train_seasonal = df_train['seasonal_vaccine']

Model - Ensemble using LGBoost, XGBoost, CatBoost and GradientBoosting 

In [60]:
#GBM model for h1n1_vaccine
gbm_model_h1n1 = GradientBoostingClassifier(n_estimators=500, max_depth=7, learning_rate=0.03,
min_samples_split=2, min_samples_leaf=1, subsample=0.9,
max_features='sqrt', random_state=42)

#GBM model for seasonal_vaccine
gbm_model_seasonal = GradientBoostingClassifier(n_estimators=500, max_depth=5, learning_rate=0.03,
min_samples_split=3, min_samples_leaf=3, subsample=0.6,
max_features='sqrt', random_state=42)

# LightGBM model for h1n1_vaccine
lgb_model_h1n1 = lgb.LGBMClassifier(device='CPU',objective='binary', learning_rate=0.03, n_estimators=500, max_depth=6,
                                    min_child_weight=2, subsample=0.9, colsample_bytree=0.5)

# XGBoost model for h1n1_vaccine
xgb_model_h1n1 = xgb.XGBClassifier(tree_method='auto',objective='binary:logistic', learning_rate=0.03, n_estimators=500, max_depth=5,
                                    min_child_weight=2, subsample=0.9, colsample_bytree=0.5)

# CatBoost model for h1n1_vaccine
cb_model_h1n1 = cb.CatBoostClassifier(objective='Logloss', learning_rate=0.03, n_estimators=1000, max_depth=6,
                                    min_child_samples=2, subsample=0.9, colsample_bylevel=0.5,
                                    early_stopping_rounds=10, verbose=0,cat_features=['education', 'race','sex', 'income_poverty',
                                                        'marital_status', 'rent_or_own', 'census_msa',
                                                        'employment_industry', 'employment_occupation',
                                                        'employment_status', 'hhs_geo_region', 'age_group'])

# LightGBM model for seasonal_vaccine
lgb_model_seasonal = lgb.LGBMClassifier(device='CPU',objective='binary', learning_rate=0.03, n_estimators=500, max_depth=5,
                                       min_child_weight=5, subsample=0.6, colsample_bytree=0.6)

# XGBoost model for seasonal_vaccine
xgb_model_seasonal = xgb.XGBClassifier(tree_method='auto',objective='binary:logistic', learning_rate=0.022, n_estimators=700, max_depth=5,
                                   min_child_weight=2, subsample=0.6, colsample_bytree=0.6)

# CatBoost model for seasonal_vaccine
cb_model_seasonal = cb.CatBoostClassifier(objective='Logloss', learning_rate=0.03, n_estimators=1000, max_depth=5,
                                   min_child_samples=5, subsample=0.6, colsample_bylevel=0.6,
                                   early_stopping_rounds=10, verbose=0,cat_features=['education', 'race','sex', 'income_poverty',
                                                        'marital_status', 'rent_or_own', 'census_msa',
                                                        'employment_industry', 'employment_occupation',
                                                        'employment_status', 'hhs_geo_region', 'age_group'])

# Train models for h1n1_vaccine
print("Training models for h1n1_vaccine...")
print("Training LGBoost Model...")
lgb_score_h1n1 = np.mean(cross_val_score(lgb_model_h1n1, X_train, y_train_h1n1, cv=5, scoring='roc_auc'))
print("LGBoost Model AUC score:", lgb_score_h1n1)
lgb_model_h1n1.fit(X_train, y_train_h1n1)
print("Training XGBoost Model...")
xgb_score_h1n1 = np.mean(cross_val_score(xgb_model_h1n1, X_train, y_train_h1n1, cv=5, scoring='roc_auc'))
print("XGBoost Model AUC score:", xgb_score_h1n1)
xgb_model_h1n1.fit(X_train, y_train_h1n1)
print("Training CatBoost Model...")
cb_score_h1n1 = np.mean(cross_val_score(cb_model_h1n1, X_train, y_train_h1n1, cv=5, scoring='roc_auc'))
print("CatBoost Model AUC score:", cb_score_h1n1)
cb_model_h1n1.fit(X_train, y_train_h1n1)
print("Training GBM Model...")
gbm_score_h1n1 = np.mean(cross_val_score(gbm_model_h1n1, X_train, y_train_h1n1, cv=5, scoring='roc_auc'))
print("GBM Model AUC score:", gbm_score_h1n1)
gbm_model_h1n1.fit(X_train, y_train_h1n1)

# Train models for seasonal_vaccine
print("Training models for seasonal_vaccine...")
print("Training LGBoost Model...")
lgb_score_seasonal = np.mean(cross_val_score(lgb_model_seasonal, X_train, y_train_seasonal, cv=5, scoring='roc_auc'))
print("LGBoost Model AUC score:", lgb_score_seasonal)
lgb_model_seasonal.fit(X_train, y_train_seasonal)
print("Training XGBoost Model...")
xgb_score_seasonal = np.mean(cross_val_score(xgb_model_seasonal, X_train, y_train_seasonal, cv=5, scoring='roc_auc'))
print("XGBoost Model AUC score:", xgb_score_seasonal)
xgb_model_seasonal.fit(X_train, y_train_seasonal)
print("Training CatBoost Model...")
cb_score_seasonal = np.mean(cross_val_score(cb_model_seasonal, X_train, y_train_seasonal, cv=5, scoring='roc_auc'))
print("CatBoost Model AUC score:", cb_score_seasonal)
cb_model_seasonal.fit(X_train, y_train_seasonal)
print("Training GBM Model...")
gbm_score_seasonal = np.mean(cross_val_score(gbm_model_seasonal, X_train, y_train_seasonal, cv=5, scoring='roc_auc'))
print("GBM Model AUC score:", gbm_score_seasonal)
gbm_model_seasonal.fit(X_train, y_train_seasonal)

overallmean_h1n1 = (lgb_score_h1n1+xgb_score_h1n1+cb_score_h1n1+gbm_score_h1n1) / 4
overallmean_seasonal = (lgb_score_seasonal+xgb_score_seasonal+cb_score_seasonal+gbm_score_seasonal) / 4
overallmean = (overallmean_h1n1+overallmean_seasonal)/2

print("AUC Scores...")
print("H1N1 = ",overallmean_h1n1)
print("Seasonal = ",overallmean_seasonal)
print("Ensemble Model = ",overallmean)

# Make predictions using each model
print("Making predictions using ensemble models...")
lgb_pred_h1n1 = lgb_model_h1n1.predict_proba(X_test)[:, 1]
xgb_pred_h1n1 = xgb_model_h1n1.predict_proba(X_test)[:, 1]
cb_pred_h1n1 = cb_model_h1n1.predict_proba(X_test)[:, 1]
gbm_pred_h1n1 = gbm_model_h1n1.predict_proba(X_test)[:, 1]

lgb_pred_seasonal = lgb_model_seasonal.predict_proba(X_test)[:, 1]
xgb_pred_seasonal = xgb_model_seasonal.predict_proba(X_test)[:, 1]
cb_pred_seasonal = cb_model_seasonal.predict_proba(X_test)[:, 1]
gbm_pred_seasonal = gbm_model_seasonal.predict_proba(X_test)[:, 1]

weighted_pred_h1n1 = ((0.2*lgb_pred_h1n1) +(0.3*xgb_pred_h1n1) + (0.25*cb_pred_h1n1) + (0.2*gbm_pred_h1n1)) / 4
weighted_pred_seasonal = ((0.25*lgb_pred_seasonal) + (0.3*xgb_pred_seasonal) + (0.2*cb_pred_seasonal) + (0.2*gbm_pred_seasonal)) / 4

# Save predictions to CSV file
print("Saving predictions to CSV file...")
ensemble_df = pd.DataFrame({'respondent_id': df_test['respondent_id'],
                            'h1n1_vaccine': weighted_pred_h1n1,
                            'seasonal_vaccine': weighted_pred_seasonal})
ensemble_df.to_csv('ensemble_predictions_final.csv', index=False)

Training models for h1n1_vaccine...
Training LGBoost Model...
LGBoost Model AUC score: 0.8713918138164658
Training XGBoost Model...
XGBoost Model AUC score: 0.8717068069809766
Training CatBoost Model...
CatBoost Model AUC score: 0.8724300478952399
Training GBM Model...
GBM Model AUC score: 0.8691264557293742
Training models for seasonal_vaccine...
Training LGBoost Model...
LGBoost Model AUC score: 0.8652760631934682
Training XGBoost Model...
XGBoost Model AUC score: 0.8653634630555794
Training CatBoost Model...
CatBoost Model AUC score: 0.8647477631594558
Training GBM Model...
GBM Model AUC score: 0.8642909969548771
AUC Scores...
H1N1 =  0.8711637811055142
Seasonal =  0.8649195715908451
Ensemble Model =  0.8680416763481796
Making predictions using ensemble models...
Saving predictions to CSV file...
